In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.stats import pearsonr

from zp_ihlt_project.config import TRAIN_DATA_WITH_FEATURES_PATH, TEST_DATA_WITH_FEATURES_PATH, FEATURE_STEPS_PATH, FEATURE_STEPS_WITH_IMPORTANCE_PATH
from zp_ihlt_project.feature_extraction import lexical_functions, preprocessing_functions, semantic_functions, ngram_functions, sentence_to_doc


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zachparent/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zachparent/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Train and Evaluate Models

This notebook demonstrates random forest regression models trained on various sets of features which were extracted in [extract_features.ipynb](extract_features.ipynb).


### Load feature data

We load the features that were extracted in [extract_features.ipynb](extract_features.ipynb), for both the training and test datasets. The test dataset is only used for final evaluation, and never for training. Validation is performed using cross-validation, as well as a train/val split.

In [2]:
all_train_df = pd.read_csv(TRAIN_DATA_WITH_FEATURES_PATH)
all_test_df = pd.read_csv(TEST_DATA_WITH_FEATURES_PATH)
feature_steps = pd.read_csv(FEATURE_STEPS_PATH)
feature_names = [col for col in all_train_df.columns if col.startswith("score_")]

## Baseline Model

We start by training a model on all features, using 5-fold cross-validation. We use a custom scorer function to calculate Pearson correlation with the gold standard scores.


In [3]:
results = []

def pearsonr_score(estimator, X, y):
    y_pred = estimator.predict(X)
    return pearsonr(y, y_pred)[0]

model = RandomForestRegressor()

scores = cross_val_score(
    model, 
    all_train_df[feature_names],
    all_train_df.gs,
    cv=5,
    n_jobs=-1,
    scoring=pearsonr_score
)
print(f"Cross-validation Pearson: {np.mean(scores)}+-{np.std(scores)}")


Cross-validation Pearson: 0.7301742753993787+-0.10645707240897379


We see that the model achieves a Pearson correlation of ~0.730, with a rather large standard deviation of ~0.10.

In [4]:
train_idx, val_idx = train_test_split(range(len(all_train_df)), test_size=0.2, random_state=42)

In [5]:
model = RandomForestRegressor()
model.fit(all_train_df.iloc[train_idx][feature_names], all_train_df.iloc[train_idx].gs)


RandomForestRegressor()

### Datasets
Sometimes we will evaluate the model on all datasets, but sometimes we will evaluate on a single corpus at a time.

In [6]:
train_datasets = all_train_df.dataset.unique().tolist()
test_datasets = all_test_df.dataset.unique().tolist()

### Evaluation Helper

We will be training models which vary by the features used, we will need to split data by dataset, and we will need to evaluate them across train/val/test. We define a helper function to get the `X` and `y` data given these parameters.

Our evaluation helper function also calculates Pearson correlation with the gold standard scores. We can see the scores for each split and each dataset, as well as the overall scores.

In [7]:
def get_X_y(df, selected_features, dataset, idx=None):
    if idx is None:
        idx = range(len(df))
    X = df.iloc[idx]
    X = X.loc[X.dataset == dataset][selected_features]
    y = df.iloc[idx]
    y = y.loc[y.dataset == dataset].gs
    return X, y

def evaluate_model_across_train_val_test(model, selected_features):
    results = []
    datasets = train_datasets
    for dataset in datasets:
        dataset_results = []
        dataset_results.append(dataset)
        
        X_train, y_train = get_X_y(all_train_df, selected_features, dataset, train_idx)
        X_val, y_val = get_X_y(all_train_df, selected_features, dataset, val_idx)
        X_test, y_test = get_X_y(all_test_df, selected_features, dataset)

        preds = model.predict(X_train)
        dataset_results.append(pearsonr(y_train, preds)[0])

        preds = model.predict(X_val)
        dataset_results.append(pearsonr(y_val, preds)[0])

        preds = model.predict(X_test)
        dataset_results.append(pearsonr(y_test, preds)[0])

        results.append(dataset_results)

    dataset_results = []
    dataset_results.append('all')
    preds = model.predict(all_train_df.iloc[train_idx][selected_features])
    dataset_results.append(pearsonr(all_train_df.iloc[train_idx].gs, preds)[0])

    preds = model.predict(all_train_df.iloc[val_idx][selected_features])
    dataset_results.append(pearsonr(all_train_df.iloc[val_idx].gs, preds)[0])

    preds = model.predict(all_test_df[all_test_df.dataset.isin(train_datasets)][selected_features])
    dataset_results.append(pearsonr(all_test_df[all_test_df.dataset.isin(train_datasets)].gs, preds)[0])

    results.append(dataset_results)

    results = pd.DataFrame(results, columns=["dataset", "train_pearson", "val_pearson", "test_pearson"])
    return results

evaluate_model_across_train_val_test(model, feature_names)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.971808,0.624113,0.640723
1,MSRvid,0.981643,0.765377,0.841203
2,SMTeuroparl,0.965313,0.783888,0.493122
3,all,0.984429,0.839584,0.808451


With no feature selection, we see that the model achieves a Pearson correlation of ~0.984 on the training set, ~0.840 on the validation set, and ~0.808 on the test set.

## Analyze Feature Importances

\>2000 features is a lot of features! It requires ~1min to train, and it may not be clear which features the model is paying attention to. A random forest regressor allows us to extract the feature importances directly and inspect the values. We can then try training and evaluating new models on a subset of features.

In [27]:
feature_importances = model.feature_importances_
ranked_feature_importance_indices = np.argsort(feature_importances)[::-1]
feature_steps['importance'] = feature_importances
feature_steps.sort_values(by='importance', ascending=False, inplace=True)
feature_steps.to_csv(FEATURE_STEPS_WITH_IMPORTANCE_PATH)
feature_steps.head(10)

,name,metric,step_0,step_1,step_2,step_3,step_4,step_5,step_6,step_7,importance
1008,score_jaccard_252,jaccard,sentence_to_doc,chunk_NEs,remove_stopwords,get_token_text,remove_non_alnum,NaN,NaN,NaN,0.174892
822,score_euclidean_205,euclidean,sentence_to_doc,get_tokens,get_stopwords,lemmatize_tokens,get_characters,get_2grams,NaN,NaN,0.099179
139,score_manhattan_34,manhattan,sentence_to_doc,get_tokens,get_token_text,get_characters,get_3grams,NaN,NaN,NaN,0.071025
1759,score_manhattan_439,manhattan,sentence_to_doc,chunk_NEs,get_stopwords,get_token_text,remove_non_alnum,get_characters,get_4grams,NaN,0.031740
184,score_jaccard_46,jaccard,sentence_to_doc,get_tokens,lemmatize_tokens,remove_non_alnum,get_3grams,NaN,NaN,NaN,0.025453
1463,score_manhattan_365,manhattan,sentence_to_doc,get_tokens,get_stopwords,get_token_text,remove_non_alnum,get_characters,get_2grams,NaN,0.022712
1210,score_euclidean_302,euclidean,sentence_to_doc,chunk_NEs,get_stopwords,lemmatize_tokens,remove_non_alnum,get_3grams,NaN,NaN,0.022582
123,score_manhattan_30,manhattan,sentence_to_doc,chunk_NEs,get_pos_tags,get_3grams,NaN,NaN,NaN,NaN,0.021108
976,score_jaccard_244,jaccard,sentence_to_doc,chunk_NEs,get_synsets,remove_non_alnum,get_characters,NaN,NaN,NaN,0.019456
603,score_manhattan_150,manhattan,sentence_to_doc,get_tokens,lemmatize_tokens,remove_non_alnum,get_characters,get_3grams,NaN,NaN,0.015462


In [30]:
print(f"Sum of top 10 feature importances: {sum(feature_steps.head(10).importance)}")
print(f"Sum of all feature importances: {sum(feature_steps.importance)}")
print(f"Number of features: {len(feature_steps)}")

Sum of top 10 feature importances: 0.5036096424161475
Sum of all feature importances: 1.0
Number of features: 2080


We see that the top 10 features account for ~50% of the total feature importances.

### Train Models on features with non-zero importances
We can try training models on a subset of the features with non-zero importances.

In [10]:
nonzero_feature_importances = feature_importances > 0
nonzero_importance_feature_indexes = ranked_feature_importance_indices[nonzero_feature_importances]
nonzero_importance_features = np.array(feature_names)[nonzero_importance_feature_indexes]
print(f"Number of features with non-zero importances: {len(nonzero_importance_features)}")
nonzero_importance_features_model = RandomForestRegressor()
nonzero_importance_features_model.fit(all_train_df.iloc[train_idx][nonzero_importance_features], all_train_df.iloc[train_idx].gs)

1248

It turns out there are 1248 features with non-zero importances, which is about 60% of the total number of features.

In [12]:
evaluate_model_across_train_val_test(nonzero_importance_features_model, nonzero_importance_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.974732,0.640058,0.638196
1,MSRvid,0.982998,0.768710,0.842473
2,SMTeuroparl,0.964952,0.795181,0.500792
3,all,0.985491,0.843870,0.809224


As expected, the difference in performance is negligible.

### Train Models on top 500 features
We can also try training models on the top 500 features.

In [13]:
top_500_features_indices = ranked_feature_importance_indices[:500]
top_500_features = np.array(feature_names)[top_500_features_indices]

In [14]:
top_500_features_model = RandomForestRegressor()
top_500_features_model.fit(all_train_df.iloc[train_idx][top_500_features], all_train_df.iloc[train_idx].gs)

RandomForestRegressor()

In [15]:
evaluate_model_across_train_val_test(top_500_features_model, top_500_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.972660,0.627474,0.646293
1,MSRvid,0.981891,0.766321,0.839633
2,SMTeuroparl,0.965670,0.791876,0.498844
3,all,0.984774,0.841066,0.808784


With only 500 features, we see a slight drop in performance, but the model takes about half the time to train.

## Evaluating Feature Groups

The methods used to extract features include lexical, semantic, and n-gram methods, along with some preprocessing steps. We can try training models on subsets of these feature groups to see if one is more important than the others.

We define a helper function to filter features by the steps used to extract them. We can specify the feature groups simply using the list of processing functions that are defined in [feature_extraction.py](../src/zp_ihlt_project/feature_extraction.py).

In [16]:
def filter_features_by_steps(
    feature_steps,
    allowed_steps,
):
    allowed_step_names = [step.__name__ for step in [sentence_to_doc, *allowed_steps]]
    step_col_names = [col for col in feature_steps.columns if col.startswith("step_")]
        
    selected_features = []
    for _, row in feature_steps.iterrows():
        if all(pd.isna(step) or step in allowed_step_names for step in row[step_col_names]):
            feature_name = row["name"]
            selected_features.append(feature_name)
    return selected_features


### Lexical Features

In [31]:
lexical_features = filter_features_by_steps(feature_steps, preprocessing_functions + lexical_functions + ngram_functions)
print(f"Number of lexical-based features: {len(lexical_features)}")

Number of lexical features: 320


In [34]:
lexical_model = RandomForestRegressor()
lexical_model.fit(all_train_df.iloc[train_idx][lexical_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(lexical_model, lexical_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.973128,0.635308,0.645310
1,MSRvid,0.979319,0.723569,0.813330
2,SMTeuroparl,0.963986,0.758461,0.525538
3,all,0.983557,0.822308,0.803948


Training using only Lexical Features and N-grams has a very slight detrimental effect on performance, but the model is still very strong. This suggests that the lexical features are an important component of the model, but not the only important component.

### Semantic Features

In [19]:
semantic_features = filter_features_by_steps(feature_steps, preprocessing_functions + semantic_functions + ngram_functions)
print(f"Number of semantic-based features: {len(semantic_features)}")

240

In [20]:
semantic_model = RandomForestRegressor()
semantic_model.fit(all_train_df.iloc[train_idx][semantic_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(semantic_model, semantic_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.965497,0.548925,0.533461
1,MSRvid,0.965918,0.759557,0.811493
2,SMTeuroparl,0.958751,0.827397,0.433018
3,all,0.976742,0.833922,0.768575


If we use only semantic features, there is a fairly large drop in performance on the test set, although the validation set performance actually improves. This suggests that semantic features may not be very good generalizable features for this task.

### N-grams

We were also curious to see if n-grams were important, so we trained a model without them.

In [21]:
without_ngrams_features = filter_features_by_steps(feature_steps, preprocessing_functions + semantic_functions + lexical_functions)
print(f"Number of n-gram-based features: {len(without_ngrams_features)}")

520

In [22]:
without_ngrams_model = RandomForestRegressor()
without_ngrams_model.fit(all_train_df.iloc[train_idx][without_ngrams_features], all_train_df.iloc[train_idx].gs)
evaluate_model_across_train_val_test(without_ngrams_model, without_ngrams_features)

,dataset,train_pearson,val_pearson,test_pearson
0,MSRpar,0.969039,0.646847,0.584065
1,MSRvid,0.978863,0.733982,0.807903
2,SMTeuroparl,0.962389,0.749972,0.465232
3,all,0.982838,0.824463,0.772221


Here, we see that while validation performance is largely the same, test performance is worse. This suggests that n-grams are a key feature that generalizes well to the test set.

## Final Model

Now we train a final model using all features and all training data, in hope of achieving the best possible performance on the full test set, including unseen datasets.

In [23]:
fully_trained_model = RandomForestRegressor()
fully_trained_model.fit(all_train_df[feature_names], all_train_df.gs)

RandomForestRegressor()

In [36]:
def evaluate_model_across_test(model, selected_features):
    results = []
    datasets = test_datasets
    for dataset in datasets:
        dataset_results = []
        dataset_results.append(dataset)

        preds = model.predict(all_test_df[all_test_df.dataset == dataset][selected_features])
        dataset_results.append(pearsonr(all_test_df[all_test_df.dataset == dataset].gs, preds)[0])

        results.append(dataset_results)

    preds = model.predict(all_test_df[selected_features])
    results.append(['all', pearsonr(all_test_df.gs, preds)[0]])
    results = pd.DataFrame(results, columns=["dataset", "test_pearson"])
    return results

test_results = evaluate_model_across_test(fully_trained_model, feature_names)
test_results

,dataset,test_pearson
0,MSRpar,0.644896
1,MSRvid,0.842881
2,SMTeuroparl,0.499411
3,OnWN,0.641075
4,SMTnews,0.439994
5,all,0.736393


In [25]:
results_to_beat = pd.DataFrame(np.array([[.683, .873, .528, .664, .493, 0.823]]).T, index=[*test_datasets, 'all'], columns=["pearson_to_beat"])
results_to_beat

,pearson_to_beat
MSRpar,0.683
MSRvid,0.873
SMTeuroparl,0.528
OnWN,0.664
SMTnews,0.493
all,0.823


# Discussion